In [42]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from pandas import DataFrame

In [43]:
client = Elasticsearch()
#we are going to extract the following dates, at the end we want to know in which hours, months ..etc, github has
#more activity.
#first we are goint to just extract the data and use dataframes to compute results.
query = Search(using=client, index="git_data_ok").source(['commit_date', 'lines_changed', 'lines_removed',
                                                             'lines_added'])
query_len_search = query.execute()
total_hits = query_len_search.hits.total
final_query = query[0:total_hits].execute()
#to retrieve all the data and avoid the 500 error we need to make the following set up in kibana.
#PUT your_index_name/_settings
#{ 
#  "max_result_window" : 500000 
#}
query_results = [[doc['commit_date'],doc['lines_changed'],doc['lines_removed'],doc['lines_added']] 
                 for doc in final_query]

In [44]:
#we create a data frame with the results.
df= DataFrame.from_records(query_results, columns = ['commit_date', 'lines_changed', 'lines_removed', 
                                                     'lines_added'])
print(df.head())
df.describe()

           commit_date  lines_changed  lines_removed  lines_added
0  2017-02-21T23:12:15              4              2            2
1  2018-01-14T13:17:17             11              1           10
2  2018-01-02T18:06:39            475            177          298
3  2017-06-27T22:41:56              2              0            2
4  2017-04-03T10:16:25             24              0           24


,lines_changed,lines_removed,lines_added
count,4.920700e+04,4.920700e+04,4.920700e+04
mean,9.491339e+02,2.393383e+02,7.097956e+02
std,9.374744e+04,2.156825e+04,9.079776e+04
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,0.000000e+00,1.000000e+00
50%,3.000000e+00,1.000000e+00,1.000000e+00
75%,2.800000e+01,4.000000e+00,1.600000e+01
max,1.951973e+07,4.318554e+06,1.951973e+07


In [45]:
#we are going to create a column with the hour.
def get_hour(timestamp):
    if 'T' in timestamp:
        return timestamp.split('T')[1][0:2]
df['hour']=df['commit_date'].apply(lambda x : get_hour(x))
df.head()

,commit_date,lines_changed,lines_removed,lines_added,hour
0,2017-02-21T23:12:15,4,2,2,23
1,2018-01-14T13:17:17,11,1,10,13
2,2018-01-02T18:06:39,475,177,298,18
3,2017-06-27T22:41:56,2,0,2,22
4,2017-04-03T10:16:25,24,0,24,10


In [46]:
#which are the utc hours when more files are changed.
print(df.groupby(['hour'])['lines_changed'].mean())
print(df.groupby(['hour'])['lines_removed'].mean())
print(df.groupby(['hour'])['lines_added'].mean())
#around mid -day in UTC it is when github register the most activity.

hour
00      81.320976
01    1039.629866
02     259.389374
03    1206.602542
04     639.579890
05     501.262411
06    3530.468182
07      50.081950
08    1328.465036
09     222.349542
10     615.606472
11     492.484245
12    1655.614265
13     352.388926
14     161.576066
15    5477.090465
16     171.829137
17     210.174572
18     166.851235
19      94.780048
20     125.008500
21     130.619699
22    1197.447809
23     378.726519
Name: lines_changed, dtype: float64
hour
00      20.084087
01     368.278908
02      11.929421
03    1068.341525
04      42.903581
05      44.127660
06    3480.027273
07      25.099066
08     375.378296
09      50.100726
10     293.811413
11     435.973198
12      34.599709
13     142.060316
14      41.078404
15      78.055420
16      35.555156
17     114.528771
18      34.710526
19      24.811298
20      33.159684
21      66.551984
22      28.461355
23     265.854512
Name: lines_removed, dtype: float64
hour
00      61.236890
01     671.350959
02     247.45

In [47]:
#now we realize the same calculation by month.
month_names = {'01':'a.January','02':'b.February','03':'c.March','04':'d.April','05':'e.May',
               '06':'f.June', '07':'g.July','08':'h.August','09':'i.September','10':'o.October',
               '11':'p.November', '12':'q.December'}
def get_month(timestamp):
    if 'T' in timestamp:
        return month_names.get(timestamp.split('-')[1][0:2],None)
df['month']=df['commit_date'].apply(lambda x : get_month(x))
df.head(10)

,commit_date,lines_changed,lines_removed,lines_added,hour,month
0,2017-02-21T23:12:15,4,2,2,23,b.February
1,2018-01-14T13:17:17,11,1,10,13,a.January
2,2018-01-02T18:06:39,475,177,298,18,a.January
3,2017-06-27T22:41:56,2,0,2,22,f.June
4,2017-04-03T10:16:25,24,0,24,10,d.April
5,2017-02-14T17:39:35,24,0,24,17,b.February
6,2017-11-22T14:45:38,197,107,90,14,p.November
7,2017-11-21T17:45:11,7,1,6,17,p.November
8,2017-09-01T08:32:07,52,0,52,08,i.September
9,2018-01-18T11:08:06,243,0,243,11,a.January


In [48]:
# which are the mnths when more files are changed.
print(df.groupby(['month'])['lines_changed'].sum())
print(df.groupby(['month'])['lines_removed'].sum())
print(df.groupby(['month'])['lines_added'].sum())
#around mid -day in UTC it is when github register the most activity.
#Surprisingly, August is the month when most lines are changed

month
a.January       1116031
b.February       477631
c.March          643827
d.April         3467135
e.May           5187496
f.June          1012042
g.July           602968
h.August       20070297
i.September     7728572
o.October       2043230
p.November      3565498
q.December       789303
Name: lines_changed, dtype: int64
month
a.January       714302
b.February      150373
c.March         389356
d.April         955097
e.May          4583955
f.June          361053
g.July          131101
h.August        133310
i.September    1355161
o.October       898436
p.November     1911321
q.December      193655
Name: lines_removed, dtype: int64
month
a.January        401729
b.February       327258
c.March          254471
d.April         2512038
e.May            603541
f.June           650989
g.July           471867
h.August       19936987
i.September     6373411
o.October       1144794
p.November      1654177
q.December       595648
Name: lines_added, dtype: int64


In [55]:
#now we are going to use real elastic search querys we are not going to play with timestamps any more.lets try to get
#the number of commits per month.
query = Search(using=client, index="git_data_ok")
query.source(['commit_date'])
query.aggs.bucket('histogram', 'date_histogram',field='commit_date', interval='month')
results = query.execute()
#we print results in a DataFrame.
Df_results = DataFrame.from_records([[document['key_as_string'],document['doc_count']] for document 
                                                         in results.to_dict()["aggregations"]
                                                           ['histogram']['buckets']],
                                     columns=['month','number_of_commits'])
Df_results

,month,number_of_commits
0,2013-02-01T00:00:00.000Z,4
1,2013-03-01T00:00:00.000Z,6
2,2013-04-01T00:00:00.000Z,7
3,2013-05-01T00:00:00.000Z,17
4,2013-06-01T00:00:00.000Z,37
5,2013-07-01T00:00:00.000Z,20
6,2013-08-01T00:00:00.000Z,13
7,2013-09-01T00:00:00.000Z,29
8,2013-10-01T00:00:00.000Z,27
9,2013-11-01T00:00:00.000Z,11


In [211]:
#now lets try to figure out which are the companies with more github activity.lets try to figure out in a single query
query = Search(using=client, index="git_data_ok")
query.aggs.bucket('company', 'terms', field='author_org_name')
query.aggs['company'].metric('lines_added', 'sum', field='lines_added')
query.aggs['company'].metric('lines_changed', 'sum', field='lines_changed')
query.aggs['company'].metric('lines_removed', 'sum', field='lines_removed')
results = query.execute()
DF_results = DataFrame.from_records([[document['key'],document['doc_count'],
                                      document['lines_added']['value'],document['lines_removed']['value'],
                                     document['lines_changed']['value']] 
                                     for document in results.to_dict()["aggregations"]["company"]['buckets']],
                                     columns=['company','commits','lines_added','lines_removed',
                                              'lines_changed'])
DF_results
#with this results we found that gmail it is the company with more github activity.

,company,commits,lines_added,lines_removed,lines_changed
0,Huawei,6862,1883313.0,2599874.0,4483187.0
1,Ericsson,6304,405930.0,312100.0,718030.0
2,Red Hat,5971,249100.0,118630.0,367730.0
3,Intel,4270,25103233.0,4790527.0,29893760.0
4,Orange,3949,434512.0,252711.0,687223.0
5,ZTE Corporation,3576,313770.0,79804.0,393574.0
6,OpenStack Foundation,3457,16387.0,8856.0,25243.0
7,ENEA AB,2987,106342.0,98703.0,205045.0
8,Unknown,2076,3048466.0,1841038.0,4889504.0
9,Linux Foundation,1649,47884.0,176895.0,224779.0


In [212]:
#but now it make sense to calculate some ratios.
DF_results['lines_added_per_commit'] = DF_results.apply(lambda row:
                                                          row['lines_added']/row['commits'], axis=1)
DF_results['lines_removed_commit'] = DF_results.apply(lambda row:
                                                          row['lines_removed']/row['commits'], axis=1)
DF_results['lines_changed_commit'] = DF_results.apply(lambda row:
                                                          row['lines_changed']/row['commits'], axis=1)
DF_results['lines_removed_per_lines_changed'] = DF_results.apply(lambda row:
                                                          row['lines_added']/row['lines_removed'], axis=1)
DF_results
#looking at this examples Red Hat  seems to be a great company to work, they have lots of commits with small changes
#(just 61 lines changed).
#also they have a good ratio of lines added per lines changed , almost two lines added per line change, this means
#their commits follow business requirements so they are not used to change the code.Also this could mean that 
#they are constantly deploying new funcionalities
#instead Huawei seems to be a company with huge changes , lots commits but with lots of lines removed in each one. Also
#they just add 0,72 lines per line change which it means they are constantly refactoring their code.

,company,commits,lines_added,lines_removed,lines_changed,lines_added_per_commit,lines_removed_commit,lines_changed_commit,lines_removed_per_lines_changed
0,Huawei,6862,1883313.0,2599874.0,4483187.0,274.455407,378.879918,653.335325,0.724386
1,Ericsson,6304,405930.0,312100.0,718030.0,64.392449,49.508249,113.900698,1.300641
2,Red Hat,5971,249100.0,118630.0,367730.0,41.718305,19.867694,61.585999,2.099806
3,Intel,4270,25103233.0,4790527.0,29893760.0,5878.977283,1121.903279,7000.880562,5.240182
4,Orange,3949,434512.0,252711.0,687223.0,110.030894,63.993669,174.024563,1.719403
5,ZTE Corporation,3576,313770.0,79804.0,393574.0,87.743289,22.316555,110.059843,3.931758
6,OpenStack Foundation,3457,16387.0,8856.0,25243.0,4.740237,2.561759,7.301996,1.850384
7,ENEA AB,2987,106342.0,98703.0,205045.0,35.601607,33.044191,68.645798,1.077394
8,Unknown,2076,3048466.0,1841038.0,4889504.0,1468.432563,886.819846,2355.252408,1.655841
9,Linux Foundation,1649,47884.0,176895.0,224779.0,29.038205,107.274106,136.312310,0.270692


In [213]:
#now lets try to see this data by developer for each developer of every organization. So we could se which are the top
#developers for each company
query = Search(using=client, index="git_data_ok")
query.aggs.bucket('company', 'terms', field='author_org_name')
query.aggs['company'].bucket('developer_name', 'terms', field='Author_name')
query.aggs['company']['developer_name'].metric('lines_added', 'sum', field='lines_added')
query.aggs['company']['developer_name'].metric('lines_changed', 'sum', field='lines_changed')
query.aggs['company']['developer_name'].metric('lines_removed', 'sum', field='lines_removed')
results = query.execute()
company_bucket = ([company_bucket['key'],company_bucket['developer_name']['buckets']] for company_bucket 
                    in results.to_dict()["aggregations"]["company"]['buckets'])

developers = []
for developer in company_bucket:
    developer_company = developer[0]
    developers.extend([[developer_name['key'],developer_company,developer_name['doc_count'],
                      developer_name['lines_added']['value'],
                      developer_name['lines_removed']['value'],
                     developer_name['lines_changed']['value'],
                      developer_name['lines_added']['value']/developer_name['doc_count'],
                      developer_name['lines_removed']['value']/developer_name['doc_count'],
                      developer_name['lines_changed']['value']/developer_name['doc_count']]
                      for developer_name in developer[1]])

DF_results = DataFrame.from_records(developers,columns=['developer_name','company','commits','lines_added',
                                                        'lines_removed','lines_changed','lines_added_per_commit',
                                                        'lines_removed_per_commit','lines_changed_per_commit'])
DF_results

    

,developer_name,company,commits,lines_added,lines_removed,lines_changed,lines_added_per_commit,lines_removed_per_commit,lines_changed_per_commit
0,MatthewLi,Huawei,682,17974.0,8475.0,26449.0,26.354839,12.426686,38.781525
1,JingLu5,Huawei,627,39966.0,7978.0,47944.0,63.741627,12.724083,76.465710
2,wulin wang,Huawei,427,8497.0,4381.0,12878.0,19.899297,10.259953,30.159251
3,jean.gaoliang@huawei.com,Huawei,419,10743.0,2039.0,12782.0,25.639618,4.866348,30.505967
4,rexlee8776,Huawei,398,31719.0,7631.0,39350.0,79.695980,19.173367,98.869347
5,m00133142,Huawei,395,8635.0,4422.0,13057.0,21.860759,11.194937,33.055696
6,Chigang(Justin),Huawei,369,14647.0,19797.0,34444.0,39.693767,53.650407,93.344173
7,Dan Xu,Huawei,326,10355.0,4584.0,14939.0,31.763804,14.061350,45.825153
8,chenshuai@huawei.com,Huawei,301,20708.0,5904.0,26612.0,68.797342,19.614618,88.411960
9,Yang (Gabriel) Yu,Huawei,293,12432.0,9767.0,22199.0,42.430034,33.334471,75.764505
